In [13]:
import textwrap, sys, os, glob, shutil
import numpy as np
import MDAnalysis as mda
from pdbfixer import PDBFixer
from openbabel import openbabel
from datetime import datetime
#OpenFF
import openff
import openff.units
import openff.toolkit
import openff.interchange
#OpenMM
from openmm.app import *
from openmm import *
from openmm.unit import *

## Steps to Prepare

In [14]:
def change_resname(pdb_file_in, pdb_file_out, resname_in, resname_out):
    """
    Changes a resname in a pdb file by changing all occurences of resname_in to resname_out
    
    """

    with open(pdb_file_in, 'r') as f:
        lines = f.readlines()
    print('Effected Lines:')
    eff_lines = [line for line in lines if resname_in in line]
    for line in eff_lines:
        print(line, "-->", line.replace(resname_in, resname_out))
    user_input = input("Confirm to make these changes [y/n] :")
    if user_input == 'y':
        lines = [line.replace(resname_in, resname_out) for line in lines]
        with open(pdb_file_out, 'w') as f:
            f.writelines(lines)
        return pdb_file_out
    else:
        print('Aborting....')
        return None

In [15]:
#Bromodomains Testing
#job_inputs = {'working_dir_name': 'bromodomains_testing',
#              'ligand_resname': 'resname JQ1',
#              'crystal_pdb_fn': '3mxf.pdb',
#              'build_modes': ['SOL', 'SOL']}
# GPCR Testing
job_inputs = {'working_dir_name': 'gpcr_join_testing',
              'ligand_resname': 'resname V4O',
              'crystal_pdb_fn': '5c1m_V4O.pdb',
              'build_modes': ['MEM', 'SOL']}

In [16]:
#If needed, produce a new file with a residue not named staring with a number.  4VO would throw error, but V4O will not (idk)
#change_resname('5c1m.pdb', '5c1m_V4O.pdb', '4VO', 'V4O')

In [22]:
class Simulation_Preparer():
    def __init__(self, job_inputs):
        #Declare Filenames
        self.job_inputs = job_inputs
        working_dir_name = job_inputs['working_dir_name']
        self.ligand_resname = job_inputs['ligand_resname']
        self.crystal_pdb_fn = job_inputs['crystal_pdb_fn']

        self.abs_work_dir = os.path.join(os.getcwd(), working_dir_name)
        if not os.path.isdir(self.abs_work_dir):
            os.mkdir(self.abs_work_dir)

    def align_two_proteins(self,
                           protein_fn1,
                           protein_fn2):
        return NotImplmentedError('RIGHT ME')
    
    def seperate_crys_using_MDA(self,
                                crys_pdb_fn: str,
                                ligand_string: str,
                                receptor_string: str = 'protein',
                                ligand_pdb_fn: str = 'ligand_crys.pdb',
                                receptor_pdb_fn: str = 'receptor_crys.pdb'):
        """
        Loads the provided crystal structure, and seperates the atoms of resname LIGAND STRING into their own file
        Arguments:
            crys_pdb_fn: crystal structure to be seperated
            ligand_string: MDAnalysis selection string for ligand
            receptor_string: MDAnalysis selection string for receptor (default 'protein')
            ligand_pdb_fn: Write path for MDAnalysis ligand (default 'ligand.pdb')
            receptor_pdb_fn: Write path for MDAnalysis receptor (default 'receptor.pdb')
        Returns:
            receptor_pdb_fn: filename of the pdb file containing the receptor only
            ligand_pdb_fn: filename of the pdb file containing the atoms from the LIGAND_STRING selection
        """
        u = mda.Universe(crys_pdb_fn)
        all_atoms = u.select_atoms('all')
        receptor = all_atoms.select_atoms(receptor_string)
        ligand = all_atoms.select_atoms(ligand_string)
        receptor.write(os.path.join(self.abs_work_dir, receptor_pdb_fn))
        ligand.write(os.path.join(self.abs_work_dir, ligand_pdb_fn))
        return os.path.join(self.abs_work_dir, receptor_pdb_fn), os.path.join(self.abs_work_dir, ligand_pdb_fn)

    def obabel_conversion(self,
                          mol_fn: str,
                          formats: list,
                          out_fn: str = 'AUTO',
                          add_Hs: bool = True,
                          rewrite_with_Hs: bool = True):
        """
        Convert a file to another format using openbabel.  Neither add_Hs, nore rewrite_with_Hs should be True if the input has hydrogens.
        Arguments:
            mol_fn: in_file_name : (THis should be infile.xxx where xxx = formats[0])
            formats: 2-list of babel formats (in, out)
            out_fn: Filename for the ligand with Hydrogens (if AUTO, it will be MOL_FN with the extension swapped to format[-1])
            add_Hs: Bool: Whether adding hydrogens to the input file should be done (default True)
            rewrite_with_Hs: Bool: Additionally rewrites the protonated ligand in the original file format (default True)
        Returns:
            str: The path of the converted file
        Example:
            obabel_conversion('ligand.pdb', ['pdb', 'sdf']) will attempt to protonate (by default) and convert a pdb file to sdf (named ligand.sdf)
        """
        #Assertion checks
        assert len(formats) == 2
        #Obabel conversion block
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats(*formats)
        mol = openbabel.OBMol()
        #Find INput
        if os.path.isfile(mol_fn):
            obConversion.ReadFile(mol, mol_fn)
        elif os.path.isfile(os.path.join(self.abs_work_dir, mol_fn)):
            obConversion.ReadFile(mol, os.path.join(self.abs_work_dir, mol_fn))
        else:
            raise FileNotFoundError('mol_fn was not found')
        #Add Hydrogens
        if add_Hs:
            mol.AddHydrogens()
        print(mol.NumAtoms(), 'Atoms', mol.NumBonds(), 'Bonds', mol.NumResidues(), 'Residues')
        #Output file name parsing
        if out_fn == 'AUTO':
            out_fn = os.path.splitext(mol_fn)[0] + '.' + formats[-1]
        #Actually writeout the protonated file in the second format
        obConversion.WriteFile(mol, out_fn)
        
        #Rewrite original format with Hydrogens (if necessary)
        if rewrite_with_Hs:
            #recursively use this function to convert from format 0 to format 0 again
            org_form_new_fn = os.path.splitext(mol_fn)[0] + '_H.' + formats[0]
            org_form_wHs_fn = self.obabel_conversion(mol_fn, [formats[0], formats[0]], out_fn=org_form_new_fn, add_Hs=True, rewrite_with_Hs=False)[0]
            return (out_fn, org_form_wHs_fn)
        else:
            return (out_fn, None)

    def protonate_with_pdb2pqr(self,
                               protein_fn: str,
                               protein_H_fn: str = None,
                               at_pH=7):
        """
        Protonates the given structure using pdb2pqr30
        Parameters:
            protein_fn: structure to be protonated
            protein_H_fn: filepath for protonated receptor (as pdb)
                          pqr output of pdb2pqr is inferred as protein_H_fn with a pqr extension instead of pdb
            at_pH: pH for protonation (default 7)
        Returns:
            2-tuple = (protein_H_fn, protein_pqr_fn); file paths (as strings) to the protonated pdb and pqr file
        """
        if protein_H_fn is None:
            protein_H_fn = os.path.splitext(protein_fn)[0] + '_H' + os.path.splitext(protein_fn)[-1]
        protein_pqr_fn = os.path.splitext(protein_H_fn)[0] + '.pqr'
        my_cmd = f'pdb2pqr30 --ff AMBER --nodebump --keep-chain --ffout AMBER --pdb-output {protein_H_fn} --with-ph {at_pH} {protein_fn} {protein_pqr_fn}'
        print('Protanting using command line')
        print(f'Running {my_cmd}')
        exit_status = os.system(my_cmd)
        print(f'Done with exit status {exit_status}')
        return protein_H_fn, protein_pqr_fn

    def run_PDBFixer(self,
                     in_pdbfile: str,
                     mode: str = "MEM",
                     out_file_fn: str = None,
                     padding = 1.5,
                     ionicStrength = 0.15):
        """
        Generates a solvated and membrane-added system (depending on MODE)
        MODE = 'MEM' for membrane and solvent
        MODE = 'SOL' for solvent only
        Parameters:
            in_pdbfile: the structure to be solvated
            mode: string: must be in ['MEM', 'SOL']
            solvated_file_fn: Filename to save solvated system; default is to add '_solvated' between the body and extension of the input file name
            padding: float or int: minimum nanometer distance between the boundary and any atoms in the input.  Default 1.5 nm = 15 A
            ionicStrength: float (not int) : molar strength of ionic solution. Default 0.15 M = 150 mmol
        Returns:
            solvated_file_fn: the filename of the solvated output
        """
        assert mode in ['MEM', 'SOL']
        fixer = PDBFixer(in_pdbfile)

        if mode == 'MEM':
            fixer.addMembrane('POPC', minimumPadding=padding * nanometer, ionicStrength=ionicStrength * molar)
        elif mode == 'SOL':
            fixer.addSolvent(padding=padding * nanometer, ionicStrength=ionicStrength * molar)

        fixer.addMissingHydrogens()
        
        # ADD PDBFixer hydrogens and parsing crystal structures (Hydrogens with pdb2pqr30 at the moment)
        if out_file_fn is None:
            out_file_fn = os.path.splitext(in_pdbfile)[0] + f'_{mode}' + os.path.splitext(in_pdbfile)[-1]
        
        with open(out_file_fn, "w") as f:
            PDBFile.writeFile(fixer.topology, fixer.positions, f)
        return out_file_fn

    def insert_molecule_and_remove_clashes(self,
                                           topology: openff.toolkit.Topology,
                                           insert: openff.toolkit.Molecule,
                                           radius: openff.units.Quantity = 1.5 * openff.units.unit.angstrom,
                                           keep: list[openff.toolkit.Molecule] = []) -> openff.toolkit.Topology:
        """
        Add a molecule to a copy of the topology, removing any clashing molecules.
    
        The molecule will be added to the end of the topology. A new topology is
        returned; the input topology will not be altered. All molecules that
        clash will be removed, and each removed molecule will be printed to stdout.
        Users are responsible for ensuring that no important molecules have been
        removed; the clash radius may be modified accordingly.
    
        Parameters
        ==========
        top
            The topology to insert a molecule into
        insert
            The molecule to insert
        radius
            Any atom within this distance of any atom in the insert is considered
            clashing.
        keep
            Keep copies of these molecules, even if they're clashing
        """
        # We'll collect the molecules for the output topology into a list
        new_top_mols = []
        # A molecule's positions in a topology are stored as its zeroth conformer
        insert_coordinates = insert.conformers[0][:, None, :]
        for molecule in topology.molecules:
            if any(keep_mol.is_isomorphic_with(molecule) for keep_mol in keep): #keep is an empty list by default
                new_top_mols.append(molecule)
                continue
            molecule_coordinates = molecule.conformers[0][None, :, :]
            diff_matrix = molecule_coordinates - insert_coordinates
    
            # np.linalg.norm doesn't work on Pint quantities 😢
            working_unit = openff.units.unit.nanometer
            distance_matrix = (np.linalg.norm(diff_matrix.m_as(working_unit), axis=-1) * working_unit)
    
            if distance_matrix.min() > radius:
                # This molecule is not clashing, so add it to the topology
                new_top_mols.append(molecule)
            else:
                print(f"Removed {molecule.to_smiles()} molecule")
    
        # Insert the ligand at the end
        new_top_mols.append(insert)
    
        # This pattern of assembling a topology from a list of molecules
        # ends up being much more efficient than adding each molecule
        # to a new topology one at a time
        new_top = openff.toolkit.Topology.from_molecules(new_top_mols)
    
        # Don't forget the box vectors!
        new_top.box_vectors = topology.box_vectors
        return new_top
        
    def generate_topologies(self,
                            structure_file: str,
                            ligand_to_insert: openff.toolkit.Molecule = None,
                            json_save_fn: str = None):
        """
        Convert the final structure files into OpenFF topologies
        Parameters:
            structure_file: the structure file to generate topology for
            ligand_to_insert: If an openff toolkit openff.toolkit.Molecule type object is given, it will be inserted into the receptor.
            save_as_json: If a filename is provided, a json file of the generated topology will be saved
        Returns:
            top: The generated topology
        """
        #Create the topology of the complex phase
        top = openff.toolkit.Topology.from_pdb(structure_file)
        #Insert the ligand into this phase and remove clashes
        if ligand_to_insert is not None:
            top = self.insert_molecule_and_remove_clashes(top, ligand_to_insert)
        if json_save_fn is not None:
            with open(json_save_fn, "w") as f:
                print(top.to_json(), file=f)
        return top

    def top2interchange(self, top: openff.toolkit.Topology, xmls: list):
        """
        Convert an OpenFF openff.toolkit.Topology into and OpenFF Interchange (Can take a long time!)
        This is the actual step where MD parameters are applied
        Parameters:
            top: the topology to be converted
        Returns:
            interchange: The interchange object which was created
        """
        sage_ff14sb = openff.toolkit.ForceField(*xmls)
        return sage_ff14sb.create_interchange(top)

    def interchange2OpenmmSim(self,
                              interchange,
                             temp):
        """
        Construct an openn simulation object from an openff interachange
        
        """
        # Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
        integrator = LangevinIntegrator(300 * kelvin, 1 / picosecond, 0.001 * picoseconds)
        # Under the hood, this creates *OpenMM* `System` and `Topology` objects, then combines them together
        simulation = interchange.to_openmm_simulation(integrator=integrator)
        # Add a reporter to record the structure every 10 steps
        dcd_reporter = DCDReporter(f"trajectory.dcd", 1000)
        simulation.reporters.append(dcd_reporter)
        simulation.reporters.append(StateDataReporter(sys.stdout, 1000, step=True, potentialEnergy=True, temperature=True))
        #Evaluate and Report pre-minimized energy
        describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Original state")
        #Minimize the structure
        simulation.minimizeEnergy()
        #Evaluate and Report post-minimized energy
        describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Minimized state")
        simulation.context.setVelocitiesToTemperature(300 * kelvin)
        return simulation

Invoke the Class

In [23]:
start = datetime.now()
self = Simulation_Preparer(job_inputs)

Using the provided crystal structure, seperate into protein and ligand components.  THis is based on selection strings, by default the "receptor" is anything under the selection string "protein" and the "ligand" is the user provided selection string for "ligand_resname"

In [24]:
receptor_path, ligand_path = self.seperate_crys_using_MDA(self.crystal_pdb_fn, self.ligand_resname)
print(receptor_path, os.path.isfile(receptor_path))
print(ligand_path, os.path.isfile(ligand_path))

/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys.pdb True
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/ligand_crys.pdb True


/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


Utilize openbabel to convert the ligand file to sdf (the prefered input format for OpenFF), while also adding hydrogens

In [25]:
ligand_sdf_path, ligand_protonated = self.obabel_conversion(ligand_path, ['pdb', 'sdf'], out_fn='AUTO', add_Hs=True, rewrite_with_Hs=True)
print(ligand_protonated, os.path.isfile(ligand_protonated))
print(ligand_sdf_path, os.path.isfile(ligand_sdf_path))

66 Atoms 72 Bonds 1 Residues
66 Atoms 72 Bonds 1 Residues
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/ligand_crys_H.pdb True
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/ligand_crys.sdf True


Define the Ligand as an OpenFF molecule

In [26]:
self.ligand = openff.toolkit.Molecule.from_file(ligand_sdf_path)
print(type(self.ligand), self.ligand.to_smiles(explicit_hydrogens=False))

<class 'openff.toolkit.topology.molecule.Molecule'> CO[C@]12CC[C@]34C[C@@]1(C)[C@H](c1ccccc1)N[C@H]2[C@@]31CCN(C)[C@@H]4Cc2ccc(O)cc21


Obtain the Ligand as an OpenMM System

In [27]:
sage_ff = openff.toolkit.ForceField('openff-2.1.0.offxml')

In [28]:
cubic_box = openff.units.Quantity(30 * np.eye(3), openff.units.unit.angstrom)
interchange = openff.interchange.Interchange.from_smirnoff(topology=[self.ligand], force_field=sage_ff, box=cubic_box)

In [29]:
ligand_positions = np.array(interchange.positions) * nanometer
ligand_sys = interchange.to_openmm_system()
ligand_top = interchange.to_openmm_topology()

/tmp/ipykernel_48567/3350557486.py:1: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  ligand_positions = np.array(interchange.positions) * nanometer


Protonate the protiein using pdb2pqr30 on the command line

In [30]:
protein_protonated, protein_pqr = self.protonate_with_pdb2pqr(receptor_path, at_pH=7)
print(protein_protonated, os.path.isfile(protein_protonated))
print(protein_pqr, os.path.isfile(protein_pqr))

Protanting using command line
Running pdb2pqr30 --ff AMBER --nodebump --keep-chain --ffout AMBER --pdb-output /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pdb --with-ph 7 /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys.pdb /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pqr


INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     1/2 of bilayer thickness:   17.2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 413 residues and 3249 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 16 mis

Done with exit status 0
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pdb True
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pqr True


INFO:Applying force field to biomolecule states.
INFO:Applying custom naming scheme (amber).
INFO:Regenerating headers.
INFO:Regenerating PDB lines.


In [31]:
openmm_ff = ForceField('amber14/protein.ff14SB.xml', 'amber14/lipid17.xml', 'wat_opc3.xml')

Solvate the receptor (without the ligand still), in Membrane or water depending on the Build Mode specified in the input file. 

This can take a while for membrane proteins...

In [32]:
self.protein_solvated = self.run_PDBFixer(protein_protonated, mode=self.job_inputs['build_modes'][0], padding=1.5, ionicStrength=0.15)
print(self.protein_solvated, os.path.isfile(self.protein_solvated))

/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H_MEM.pdb True


Solvate the ligand in it's own small box (should it be needed as a second phase of FE calculation)

In [33]:
#ligand_solvated = self.run_PDBFixer(ligand_protonated, mode=self.job_inputs['build_modes'][1], padding=1.5, ionicStrength=0.15)
#print(ligand_solvated, os.path.isfile(ligand_solvated))

In [34]:
pdb = PDBFile(self.protein_solvated)
receptor_top, receptor_positions = pdb.getTopology(), pdb.getPositions()
receptor_sys = openmm_ff.createSystem(receptor_top, nonbondedMethod=PME)

In [35]:
box_vectors = receptor_sys.getDefaultPeriodicBoxVectors()

translate = Quantity(np.array((box_vectors[0].x,
                               box_vectors[1].y,
                               box_vectors[2].z))/2,
                     unit=nanometer)

In [36]:
#Write Out Receptor Only system (for testing)
with open('test_translate_reconly.pdb', 'w') as f:
    PDBFile.writeFile(receptor_top, receptor_positions + translate, f)
    
with open('test_translate_reconly.xml', 'w') as f:
    f.write(XmlSerializer.serialize(receptor_sys))

In [37]:
def join_two_topologies(tops: tuple, poss: tuple):
    """
    Joins two topologies by adding the first to the second
    Parameters:
        tops: A two-tuple of openmm Topologies
    Return
    """
    assert len(tops) == 2 and len(poss) == 2
    modeller = Modeller(tops[0], poss[0])
    modeller.add(tops[1], poss[1])
    return modeller.topology, modeller.positions

def describe_system(sys: System):
    box_vecs = sys.getDefaultPeriodicBoxVectors()
    print('Box Vectors')
    [print(box_vec) for box_vec in box_vecs]
    forces = sys.getForces()
    print('Forces')
    [print(force) for force in forces]
    num_particles = sys.getNumParticles()
    print(num_particles, 'Particles')

def describe_state(state: State, name: str = "State"):
    max_force = max(np.sqrt(v.x**2 + v.y**2 + v.z**2) for v in state.getForces())
    print(f"{name} has energy {round(state.getPotentialEnergy()._value, 2)} kJ/mol ",
          f"with maximum force {round(max_force, 2)} kJ/(mol nm)")

def join_two_systems(sys1: System, sys2: System):
    """
    Joins Two Openmm systems by adding the elements of system 1 to system 2
    Intended use in this notebook is join_two_systems(ligand_sys, receptor_sys)
    
    Parameters:
        sys1 - The openmm system to be added to sys2
        sys2 - The openmm system to have sys1 added to
    Returns
        System - The combined system
    """
    #Particles
    new_particle_indices = []
    for i in range(sys1.getNumParticles()):
        new_particle_indices.append(sys2.addParticle(sys1.getParticleMass(i)))
    
    #Contstraints (mostly wrt hydrogen distances)
    for i in range(sys1.getNumConstraints()):
        params = sys1.getConstraintParameters(i)
        params[0] = new_particle_indices[params[0]]
        params[1] = new_particle_indices[params[1]]
        sys2.addConstraint(*params)
    
    #NonBonded
    sys1_force_name = 'Nonbonded force'
    sys2_force_name = 'NonbondedForce'
    
    force_ind = [force.getName() for force in sys1.getForces()].index(sys1_force_name)
    sys1_force = sys1.getForces()[force_ind]
    
    force_ind = [force.getName() for force in sys2.getForces()].index(sys2_force_name)
    sys2_force = sys2.getForces()[force_ind]
    
    for i in range(sys1_force.getNumParticles()):
        params = sys1_force.getParticleParameters(i)
        sys2_force.addParticle(*params)
    
    for i in range(sys1_force.getNumExceptions()):
        params = sys1_force.getExceptionParameters(i)
        params[0] = new_particle_indices[params[0]]
        params[1] = new_particle_indices[params[1]]
        sys2_force.addException(*params)

    #Torsion
    sys1_force_name = 'PeriodicTorsionForce'
    sys2_force_name = 'PeriodicTorsionForce'
    
    force_ind = [force.getName() for force in sys1.getForces()].index(sys1_force_name)
    sys1_force = sys1.getForces()[force_ind]
    
    force_ind = [force.getName() for force in sys2.getForces()].index(sys2_force_name)
    sys2_force = sys2.getForces()[force_ind]
    
    for i in range(sys1_force.getNumTorsions()):
        params = sys1_force.getTorsionParameters(i)
        params[0] = new_particle_indices[params[0]]
        params[1] = new_particle_indices[params[1]]
        params[2] = new_particle_indices[params[2]]
        params[3] = new_particle_indices[params[3]]
        sys2_force.addTorsion(*params)

    #Angle
    sys1_force_name = 'HarmonicAngleForce'
    sys2_force_name = 'HarmonicAngleForce'
    
    force_ind = [force.getName() for force in sys1.getForces()].index(sys1_force_name)
    sys1_force = sys1.getForces()[force_ind]
    
    force_ind = [force.getName() for force in sys2.getForces()].index(sys2_force_name)
    sys2_force = sys2.getForces()[force_ind]
    
    for i in range(sys1_force.getNumAngles()):
        params = sys1_force.getAngleParameters(i)
        params[0] = new_particle_indices[params[0]]
        params[1] = new_particle_indices[params[1]]
        params[2] = new_particle_indices[params[2]]
        sys2_force.addAngle(*params)

    #Bond
    sys1_force_name = 'HarmonicBondForce'
    sys2_force_name = 'HarmonicBondForce'
    
    force_ind = [force.getName() for force in sys1.getForces()].index(sys1_force_name)
    sys1_force = sys1.getForces()[force_ind]
    
    force_ind = [force.getName() for force in sys2.getForces()].index(sys2_force_name)
    sys2_force = sys2.getForces()[force_ind]
    
    for i in range(sys1_force.getNumBonds()):
        params = sys1_force.getBondParameters(i)
        params[0] = new_particle_indices[params[0]]
        params[1] = new_particle_indices[params[1]]
        sys2_force.addBond(*params)
    
    return sys2

In [38]:
comp_top, comp_positions = join_two_topologies((receptor_top, ligand_top), (receptor_positions, ligand_positions))

com_positions needs to be moved by the PeriodicBoxVectors/2 in each direction, as the origin is the center of the OPM structure

In [39]:
comp_sys = join_two_systems(ligand_sys, receptor_sys)

In [40]:
integrator = LangevinIntegrator(300 * kelvin, 1/picosecond, 0.001 * picosecond)

In [41]:
simulation = Simulation(comp_top, receptor_sys, integrator)
simulation.context.setPositions(comp_positions + translate)

In [42]:
#Evaluate and Report pre-minimized energy
describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Original state")
end = datetime.now() - start
print(f'Time to build this simulation: {end}')

Original state has energy -1800778.36 kJ/mol  with maximum force 8071191.92 kJ/(mol nm)
Time to build this simulation: 0:04:08.874151


In [43]:
#Write Out (and try minimizing on a GPU)
with open('test_translate_result.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), f)
    
with open('test_translate_result.xml', 'w') as f:
    f.write(XmlSerializer.serialize(simulation.system))